In [13]:
from aiida import load_profile
aiida_profile = load_profile()
print(f'profil name: {aiida_profile.name}')

## Loading the some require packages and module
from aiida_kkr.workflows import (combine_imps_wc, kkr_flex_wc,
                                 kkr_imp_wc, kkr_imp_sub_wc
                                )
from aiida_kkr.calculations import KkrCalculation, KkrimpCalculation
from aiida.orm import (Group, load_group, load_node, Dict, Code, QueryBuilder, WorkChainNode
                       , CalcJobNode)
from aiida.transports import transport
from aiida_kkr.tools import kkrparams
from aiida.engine import submit

profil name: default


In [14]:
#Section-1: delete node from the database as well as remote work directory
### A function to delete the data of calculation output of calcjob list (pks) from the remote dir.

# later add it with the del_node  function
# This is successfully done

# please note that it is tested for one calc list
def delete_remote_workdir(pks, verbosity=0, dry_run= True):
    from aiida.common import exceptions, NotExistent
    from aiida.orm import load_node
    from aiida.orm import computers
    import sys
    # TODO : add the verbosity as discused here 
    # https://aiida.readthedocs.io/projects/aiida-core/en/v1.5.0/_modules/aiida/manage/database/delete/nodes.html

    """
    :param pks: calc node list
    
    """
    removed_path_list = [] # The part of the path will be deleted
    remote_path_list = []  # The original path
    updated_path_list = [] # After removing the part of the path
    loadable_list = [] # To load the node and save it loadable_list
    loaded_node_list = []
    # To check the loadable calcjob list
    for pk in pks:
        try:
            loaded_node = load_node(pk)
        except exceptions.NotExistent:
            print('This is calcjob node'.format(pk))
            loaded_node = pk
            loaded_node_list.append(loaded_node)
            #             sys.exit()
        else:
            loaded_node_list.append(loaded_node)
    # Computer data
    
   
    for node in loaded_node_list:
        load_pk = node
        # computer data
        computer = load_pk.computer
        computer_name = computer.label
        print(computer_name)
        try:
            remote_path = load_pk.get_remote_workdir()
            
        except NotExistent as ex:
            print(f'Node (label,pk) ({node.label, node.pk}) does not have remote workdir. with exception : {ex}')
        if isinstance(remote_path, str):
            remote_path_list.append(remote_path)
            delete_folder = remote_path.split('/')[-1]
            removed_path_list.append(delete_folder)

            new_remote_path = remote_path.replace(remote_path.split('/')[-1], '')
            updated_path_list.append(new_remote_path)
        
        
    if dry_run or verbosity == 3:

        for i, paths in enumerate(zip(remote_path_list, updated_path_list)):
            print('Before the delation the original path list : {}\n'.format(paths[0]))
            print('After deletion the modefied or updated path : {}'.format(paths[1]))
    if verbosity == 3 or verbosity == 2:
        val = input("Are you agree to clean the remote workdir (y/n) : ")
    else:
        val = 'y'
    if str(val)=='y' or str(val)=='Y':
        if not dry_run:
            for remote_path in remote_path_list:
                try:
                    # Open the connection to the remote folder/dir via transport
                    computer_transport = computer.get_transport()
                    is_transport_open = computer_transport.is_open
                    if not is_transport_open:
                        computer_transport.open()
                    computer_transport.rmtree(remote_path)
                except IOError as ex:
                    print('Uable to open the computer transport: ', ex)
    else:
        print('Nothing to clean from the remote workdir!')
# section-5:delete node from the database as well as remote work directory
## It returns all the calcjob from a WC node
def find_calcJob(pk_or_node, debug=True):
    
    calcjob_node_list=[]
    wc_node_list = []
    try:
        if isinstance( pk_or_node, int):
            if debug:
                print('This is pk')
            node = load_node(pk_or_node)
        else:
            if debug:
                print('This is node.')
            node= pk_or_node
    except:
        print('{} is nither node ID nor aiida_node. '.format(pk_or_node))
        
    ## Use the get_calcjob_wc to get descendent calcjob list and  wc list
    calc_list, wc_list = get_calcjob_wc(node)
    calcjob_node_list += calc_list
    
    while len(wc_list)!=0:
        new_wc_list = []

        for i in wc_list[:]:
            calc_list, wc_list = get_calcjob_wc(i)
            new_wc_list += wc_list
            calcjob_node_list += calc_list
            
        wc_list = new_wc_list

    return calcjob_node_list

## This function returns calcjob_list and wc_list from a wc or calcjob node   
def get_calcjob_wc(node):
    """
    :param: node
    :return: workchain node list and calcjob node list
    """ 
    from aiida.orm import CalcJobNode, WorkChainNode
    wc = []
    calc_job = []
    
    if node.node_type == 'process.workflow.workchain.WorkChainNode.':
        
    # here all outgoing worchain node
        out_going_wc = node.get_outgoing(node_class=WorkChainNode).all()
        wc = [i.node for i in out_going_wc[:]]
        
    # here all outgoing calcjob node
        out_going_calc = node.get_outgoing(node_class=CalcJobNode).all()
        calc_job = [i.node for i in out_going_calc[:]]
                    
    elif node.node_type == 'process.calculation.calcjob.CalcJobNode.':
        calc_job.append(node)
    
    return calc_job, wc

# This is the final del_node_function. Using this function for any specific wc node the node from the 
# Db as well as the calcjob data from the remote workdir can be deleted.
def del_node(node_pks, dry_run=True, verbosity=3, debug=True, only_remote_dir=False,
            only_database=False):
    """
    1. This function will delete the node data from the database and also from the remote_dir
    
    :params node_pks: (list) list of workchain to delete from database as well as from remote workdir
    :param verbosity: 0 prints nothing.  This is for workdir and wc
                      1 prints just sums and total.   This is for workdir but not for wc
                      2 prints indivisual nodes.  This is for workdir and wc
    :param dry_run: Do not delete anything just show the status as in the verbosity given
    """
    from aiida.orm import load_node
    from aiida.manage.database.delete.nodes import delete_nodes
    
        
    calcjobs_list = []
    pks_given = []
    for i in node_pks:    
        try:
            if isinstance( i, int):
                if debug:
                    print('This might be pk or uiid')
                node = load_node(i)
            else:
                if debug:
                    print('This might be a node.')
                node= i
        except:
            print('{} is nither node ID nor aiida_node. '.format(i))
        
        pks_given.append(node.pk)
        
        calcjobs = find_calcJob(node, debug)
        calcjobs_list += calcjobs
        print('calcjob list : ', calcjobs_list,)
    if only_remote_dir:
        delete_remote_workdir(calcjobs_list, verbosity=verbosity, dry_run=dry_run)
    if only_database:
        delete_nodes(pks_given, verbosity=verbosity, dry_run=dry_run,force=False)
    

In [21]:
del_node([105762, 105771, 105788, 105797], dry_run= False, verbosity= 3, 
            debug= True, only_remote_dir= True, only_database= True)

This might be pk or uiid
This is node.
calcjob list :  [<CalcJobNode: uuid: f3bbfd51-5d4a-4625-a3e1-8c80ea4a4bd2 (pk: 105767) (aiida.calculations:kkr.kkr)>]
This might be pk or uiid
This is node.
calcjob list :  [<CalcJobNode: uuid: f3bbfd51-5d4a-4625-a3e1-8c80ea4a4bd2 (pk: 105767) (aiida.calculations:kkr.kkr)>, <CalcJobNode: uuid: e83fe44f-02b7-4f25-b568-74b568770ef6 (pk: 105776) (aiida.calculations:kkr.kkr)>]
This might be pk or uiid
This is node.
calcjob list :  [<CalcJobNode: uuid: f3bbfd51-5d4a-4625-a3e1-8c80ea4a4bd2 (pk: 105767) (aiida.calculations:kkr.kkr)>, <CalcJobNode: uuid: e83fe44f-02b7-4f25-b568-74b568770ef6 (pk: 105776) (aiida.calculations:kkr.kkr)>, <CalcJobNode: uuid: 86c349bf-51ca-4a39-bffb-7d4aef7e2f9d (pk: 105793) (aiida.calculations:kkr.kkr)>]
This might be pk or uiid
This is node.
calcjob list :  [<CalcJobNode: uuid: f3bbfd51-5d4a-4625-a3e1-8c80ea4a4bd2 (pk: 105767) (aiida.calculations:kkr.kkr)>, <CalcJobNode: uuid: e83fe44f-02b7-4f25-b568-74b568770ef6 (pk: 105776)

Are you agree to clean the remote workdir (y/n) :  y


I will delete 48 nodes
The nodes I will delete:
   66b59df2-3dbe-49a9-8ac7-3c8dde6c85be 105762 WorkChainNode Dos calculation for host Bi_2Te_3
   36371352-d687-492a-96e8-406c16cc6c6b 105764 CalcFunctionNode update_params_wf
   c5c54764-4da6-4e78-ab0b-bfa78007d0fb 105765 Dict updated KKR parameters
   f3bbfd51-5d4a-4625-a3e1-8c80ea4a4bd2 105767 CalcJobNode KKR DOS calc.
   e55fcf2d-14fe-4b2b-be8e-0d69294941db 105768 RemoteData 
   cbc39a36-aaa8-4f9f-ab24-d0af09fc7480 105771 WorkChainNode Dos calculation for host Bi_2Te_3
   5fb03a74-230b-499f-958d-ba958b4d95f0 105773 CalcFunctionNode update_params_wf
   d4e143ad-c001-4377-a47a-585aed9144c1 105774 Dict updated KKR parameters
   e83fe44f-02b7-4f25-b568-74b568770ef6 105776 CalcJobNode KKR DOS calc.
   3513ae1d-a66d-4a62-8dd8-5d35f5d59182 105777 RemoteData 
   8b35fa37-b171-46a4-ab34-ef6fa59d5514 105778 FolderData 
   77ff85e0-c482-4b00-b5da-f74ce22928ad 105779 Dict 
   2bb9ba7a-c485-46fe-b2e0-6412182987d4 105780 CalcFunctionNode parse_dosf

 y


Starting node deletion...
Nodes deleted from database, deleting files from the repository now...
Deletion completed.
